# WHERE SHOULD I OPEN A RESTAURANT?

I AM PLANNING TO OPEN A RESTAURANT NEAR MY HOUSE(WITHIN A 10KM RADIUS). I AM LOOKING FOR THE RIGHT CUISINE AND THE RIGHT LOCATION SO THAT I CAN GAIN HUGE PROFITS, BUT HOW CAN I CHOOSE?

SO WE FIRST GET THE DATA OF THE RESTAURANTS IN BANGALORE. WE WILL EXAMINE THE DIFFERENT CUISINES IN EACH NEIGHBOR CLUSTER BASED ON THEIR RATING AND NUMBER OF RESTAURANTS. THEN WE WILL CHOOSE THE CLUSTER WHICH HAS THE LOWEST AVERAGE RESTAURANT RATING AND LEAST NUMBER OF RESTAURANTS SERVING THE HIGHEST RATED CUISINE AS OUR RESULTANT CLUSTER USING SOME WEIGHTED METHODS.

In [64]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
import requests
from geopy.geocoders import Nominatim
import json
from pandas import json_normalize

In [14]:
client_id = "P1BRDUNBKGVA44LZNLSJINAVHF52WT5IBWFUM5YNTYKJR2V1"
client_secret = "5QIX2Y4O2EHPH51USGIS0F32VLB0EGH4QZTYQKCVVPWOQS2J"

In [154]:
#house coordinates
latitude , longitude = 13.010698, 77.761386

In [155]:
latitude , longitude

(13.010698, 77.761386)

In [156]:
bl_map = folium.Map(location = [latitude,longitude],zoom_start = 12)

In [157]:
bl_map

In [104]:
#fetching the json file the restaurants in new york

In [ ]:
#first let us make a list of the top cuisines in bangalore
cuisines = ['Andhra','Dhaba','Karnataka','Kerala',
            'North Indian','Italian','Udupi''Dosa Place']
cat_id = ['54135bf5e4b08f3d2429dfe5','54135bf5e4b08f3d2429dfe1','54135bf5e4b08f3d2429dfeb','54135bf5e4b08f3d2429dfed',
          '54135bf5e4b08f3d2429dfdd','4bf58dd8d48988d110941735','54135bf5e4b08f3d2429dfec','54135bf5e4b08f3d2429dfe3']

In [182]:
url = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}".format(client_id, client_secret, latitude, longitude,20200404,"54135bf5e4b08f3d2429dfed", 10000,100)

In [183]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eaffdf3aba297001b5c90c6'},
 'response': {'venues': [{'id': '56e66bd9498e8f8e722e2a00',
    'name': 'Kappa',
    'location': {'lat': 12.98339,
     'lng': 77.65773,
     'labeledLatLngs': [{'label': 'display',
       'lat': 12.98339,
       'lng': 77.65773}],
     'distance': 11646,
     'cc': 'IN',
     'country': 'India',
     'formattedAddress': ['India']},
    'categories': [{'id': '54135bf5e4b08f3d2429dfed',
      'name': 'Kerala Restaurant',
      'pluralName': 'Kerala Restaurants',
      'shortName': 'Kerala',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1588592518',
    'hasPerk': False},
   {'id': '549fb8c6498e5892fb9c3b8a',
    'name': 'Cheenavala',
    'location': {'address': 'No 104, Main Road,',
     'lat': 12.966435104932492,
     'lng': 77.7493684737628,
     'labeledLatLngs': [{'label': 'display',
       'lat': 12.9664351049324

In [184]:
venues = results['response']['venues']

In [185]:
df = json_normalize(venues)
df

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.address,location.postalCode,location.city,location.state,location.crossStreet
0,56e66bd9498e8f8e722e2a00,Kappa,"[{'id': '54135bf5e4b08f3d2429dfed', 'name': 'K...",v-1588592518,False,12.983390,77.657730,"[{'label': 'display', 'lat': 12.98339, 'lng': ...",11646,IN,India,[India],NaN,NaN,NaN,NaN,NaN
1,549fb8c6498e5892fb9c3b8a,Cheenavala,"[{'id': '54135bf5e4b08f3d2429dfed', 'name': 'K...",v-1588592518,False,12.966435,77.749368,"[{'label': 'display', 'lat': 12.96643510493249...",5096,IN,India,"[No 104, Main Road,, Whitefield 560066, Karnāt...","No 104, Main Road,",560066,Whitefield,Karnātaka,NaN
2,56d19fbdcd1031ad78a36e88,Kuttanad Restaurant,"[{'id': '54135bf5e4b08f3d2429dfed', 'name': 'K...",v-1588592518,False,12.984643,77.675705,"[{'label': 'display', 'lat': 12.984643, 'lng':...",9735,IN,India,[India],NaN,NaN,NaN,NaN,NaN
3,560f587f498effac79b2199c,4 You,"[{'id': '54135bf5e4b08f3d2429dfed', 'name': 'K...",v-1588592518,False,13.019828,77.674965,"[{'label': 'display', 'lat': 13.01982784271240...",9428,IN,India,[India],NaN,NaN,NaN,NaN,NaN
4,5bb092b4a2c00b002cef23ee,Adukkala,"[{'id': '54135bf5e4b08f3d2429dfed', 'name': 'K...",v-1588592518,False,13.019649,77.674110,"[{'label': 'display', 'lat': 13.019649, 'lng':...",9518,IN,India,"[Bangalore 560016, Karnātaka, India]",NaN,560016,Bangalore,Karnātaka,NaN
5,570a7120498e7b1197633828,Kidilam,"[{'id': '54135bf5e4b08f3d2429dfed', 'name': 'K...",v-1588592518,False,12.983884,77.739990,"[{'label': 'display', 'lat': 12.983884, 'lng':...",3780,IN,India,[India],NaN,NaN,NaN,NaN,NaN
6,5c4ac58bd807ee002c970926,Savoury Restaurant,"[{'id': '54135bf5e4b08f3d2429dfed', 'name': 'K...",v-1588592518,False,12.985443,77.672920,"[{'label': 'display', 'lat': 12.985443, 'lng':...",9998,IN,India,"[Bangalore 560093, Karnātaka, India]",NaN,560093,Bangalore,Karnātaka,NaN
7,5bc579ba1af852002c3334d0,Dhe Thattukada,"[{'id': '54135bf5e4b08f3d2429dfed', 'name': 'K...",v-1588592518,False,13.028205,77.679405,"[{'label': 'display', 'lat': 13.028205, 'lng':...",9102,IN,India,"[Bangalore 560016, Karnātaka, India]",NaN,560016,Bangalore,Karnātaka,NaN
8,59e60c869fca56044ae53303,Adukkala - Adige Mane Restaurant,"[{'id': '54135bf5e4b08f3d2429dfed', 'name': 'K...",v-1588592518,False,13.020512,77.656527,"[{'label': 'display', 'lat': 13.020512, 'lng':...",11425,IN,India,[Opposite Sobha Supermarket (Horamavu Main Roa...,Opposite Sobha Supermarket,560043,Bengaluru South,Karnātaka,Horamavu Main Road
9,59a2fcd483e3805cc2b8e934,Malabar Mini Restaurant,"[{'id': '54135bf5e4b08f3d2429dfed', 'name': 'K...",v-1588592518,False,12.967806,77.719030,"[{'label': 'display', 'lat': 12.967806, 'lng':...",6626,IN,India,"[Bangalore 560037, Karnātaka, India]",NaN,560037,Bangalore,Karnātaka,NaN
